# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846736109937                   -0.70    4.0
  2   -7.852277286690       -2.26       -1.53    1.0
  3   -7.852645974077       -3.43       -2.53    3.5
  4   -7.852646677653       -6.15       -3.33    2.8
  5   -7.852646686116       -8.07       -4.75    1.2
  6   -7.852646686728       -9.21       -5.37    3.5
  7   -7.852646686730      -11.74       -6.06    1.0
  8   -7.852646686730      -12.69       -7.43    2.2
  9   -7.852646686730   +    -Inf       -7.73    3.0
 10   -7.852646686730   +    -Inf       -8.39    1.0
 11   -7.852646686730      -15.05       -9.70    2.5
 12   -7.852646686730   +  -14.57       -8.62    1.0
 13   -7.852646686730      -14.45       -8.37    1.0
 14   -7.852646686730   +  -14.75       -8.85    1.0
 15   -7.852646686730      -14.75       -8.65    1.0
 16   -7.852646686730   +  -14.57       -8.70    1.0
 17   -7.852646686730   +  -14.57       -9.64 

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846826185617                   -0.70           4.0
  2   -7.852525872386       -2.24       -1.62   0.80    5.0
  3   -7.852610756907       -4.07       -2.78   0.80    5.2
  4   -7.852646569771       -4.45       -3.55   0.80    7.2
  5   -7.852646683550       -6.94       -4.49   0.80    8.8
  6   -7.852646686666       -8.51       -5.10   0.80   10.2
  7   -7.852646686725      -10.24       -6.19   0.80   11.5
  8   -7.852646686730      -11.28       -7.25   0.80   13.5
  9   -7.852646686730      -13.52       -7.68   0.80   15.8
 10   -7.852646686730      -14.27       -9.53   0.80   15.8
 11   -7.852646686730   +  -14.45       -9.74   0.80   19.5
 12   -7.852646686730   +    -Inf      -10.31   0.80   19.5
 13   -7.852646686730      -14.27      -11.82   0.80   20.0
 14   -7.852646686730   +  -14.45      -11.87   0.80   23.5
 15   -7.852646686730   +  -14.75      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.413460e+01     3.037528e+00
 * time: 0.5841071605682373
     1     8.892430e-01     1.936676e+00
 * time: 0.8692631721496582
     2    -1.867267e+00     2.353001e+00
 * time: 0.8995821475982666
     3    -3.869565e+00     1.906856e+00
 * time: 0.9437651634216309
     4    -5.299142e+00     1.649818e+00
 * time: 0.9869821071624756
     5    -6.824661e+00     8.529749e-01
 * time: 1.0297811031341553
     6    -7.356323e+00     3.818479e-01
 * time: 1.0730690956115723
     7    -7.539289e+00     1.531567e-01
 * time: 1.1031250953674316
     8    -7.651837e+00     1.540889e-01
 * time: 1.3537781238555908
     9    -7.756935e+00     1.328065e-01
 * time: 1.3837950229644775
    10    -7.812454e+00     9.057374e-02
 * time: 1.4142351150512695
    11    -7.838072e+00     5.206039e-02
 * time: 1.4444360733032227
    12    -7.845638e+00     4.708996e-02
 * time: 1.4751110076904297
    13    -7.848876e+00     3.492065e-02
 * time: 1.5051131248

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846795725225                   -0.70    4.0
  2   -7.852307397227       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686677                   -2.54
  2   -7.852646686730      -10.28       -5.79
  3   -7.852646686730   +    -Inf      -12.22


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 6.447116994467486e-13
|ρ_newton - ρ_scfv| = 2.5260624653536675e-14
|ρ_newton - ρ_dm|   = 1.0798539684111054e-9
